In [1]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack, hstack
import fitsio
from astropy.io import fits
import healpy as hp
from astropy import wcs

from scipy.ndimage.filters import gaussian_filter
from pathlib import Path
from multiprocessing import Pool

from scipy import stats

In [2]:
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [9]:
image_dir = '/global/cfs/cdirs/cosmo/staging/'
mask_dir = '/global/cfs/cdirs/desi/users/rongpu/dr9/90prime_moth_mask/20160603'

surveyccd_path = '/global/project/projectdirs/cosmo/work/legacysurvey/dr9m/survey-ccds-90prime-dr9.fits.gz'

# image_path_list = glob.glob(os.path.join(image_dir, '*ooi*.fits.fz'))
ccd = Table(fitsio.read(surveyccd_path))

In [10]:
mask = ccd['ccd_cuts']==0
ccd = ccd[mask]

In [11]:
# Only keep unique exposures
_, idx = np.unique(ccd['expnum'], return_index=True)
ccd = ccd[idx]

In [12]:
ccdnum_list = [1, 2, 3, 4]
ccd_ra = [0.2813, 0.2813, -0.2813, -0.2813]
ccd_dec = [0.263, -0.263, 0.263, -0.263]

binsize = 4
pix_size = 0.454/3600*binsize
vrange = 1.

In [17]:
expnum_list = [75430073, 75430074, 75430075, 75430076, 75430077, 75430078, 75430079, 75430080, 75430082, 75430083, 75430084, 75430087, 75430088, 75430089, 75430090, 75430091, 75430092, 75430093, 75430094, 75430095, 75430096, 75430097, 75430098, 75430099, 75430100, 75430101, 75430102, 75430103, 75430104, 75430105, 75430106, 75430107, 75430108, 75430109, 75430110, 75430111, 75430112, 75430113, 75430114, 75430115, 75430116, 75430117, 75430118, 75430119, 75430120, 75430121, 75430122, 75430123, 75430124, 75430125, 75430126]
print(len(expnum_list))

51


In [14]:
idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/results_20160603.pdf') as pdf:

    for ccd_index in idx:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

        ######################## Original mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(image_dir, fn_ood)
        
        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()
        
        ######################## New mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(mask_dir, fn_ood)
        
        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

9082 r 75430072
9083 r 75430073
9084 r 75430074
9085 r 75430075
9086 r 75430076
9087 r 75430077
9088 r 75430078
9089 r 75430079
9090 r 75430080
9091 r 75430082
9092 r 75430083
9093 r 75430084
9094 r 75430087
9095 r 75430088
9096 r 75430089
9097 r 75430090
9098 r 75430091
9099 r 75430092
9100 r 75430093
9101 r 75430094
9102 r 75430095
9103 r 75430096
9104 r 75430097
9105 r 75430098
9106 r 75430099
9107 r 75430100
9108 r 75430101
9109 r 75430102
9110 r 75430103
9111 r 75430104
9112 r 75430105
9113 r 75430106
9114 r 75430107
9115 r 75430108
9116 r 75430109
9117 r 75430110
9118 r 75430111
9119 r 75430112
9120 r 75430113
9121 r 75430114
9122 r 75430115
9123 r 75430116
9124 r 75430117
9125 r 75430118
9126 r 75430119
9127 r 75430120
9128 r 75430121
9129 r 75430122
9130 r 75430123
9131 r 75430124
9132 r 75430125
9133 r 75430126


------
## Compare old and new mask

In [18]:
idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/compare_20160603.pdf') as pdf:

    for ccd_index in idx:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()
        
        ######################## compare mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path_old = os.path.join(image_dir, fn_ood)
        mask_path_new = os.path.join(mask_dir, fn_ood)
        
        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood_old = fits.getdata(mask_path_old, extname='ccd'+str(ccdnum))
            ood_new = fits.getdata(mask_path_new, extname='ccd'+str(ccdnum))
            img[ood_old!=ood_new] = 1.
            img[ood_old==ood_new] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

9083 r 75430073
9084 r 75430074
9085 r 75430075
9086 r 75430076
9087 r 75430077
9088 r 75430078
9089 r 75430079
9090 r 75430080
9091 r 75430082
9092 r 75430083
9093 r 75430084
9094 r 75430087
9095 r 75430088
9096 r 75430089
9097 r 75430090
9098 r 75430091
9099 r 75430092
9100 r 75430093
9101 r 75430094
9102 r 75430095
9103 r 75430096
9104 r 75430097
9105 r 75430098
9106 r 75430099
9107 r 75430100
9108 r 75430101
9109 r 75430102
9110 r 75430103
9111 r 75430104
9112 r 75430105
9113 r 75430106
9114 r 75430107
9115 r 75430108
9116 r 75430109
9117 r 75430110
9118 r 75430111
9119 r 75430112
9120 r 75430113
9121 r 75430114
9122 r 75430115
9123 r 75430116
9124 r 75430117
9125 r 75430118
9126 r 75430119
9127 r 75430120
9128 r 75430121
9129 r 75430122
9130 r 75430123
9131 r 75430124
9132 r 75430125
9133 r 75430126


------

In [19]:
expnum_list = [75430048, 75430049, 75430050, 75430051, 75430052, 75430053, 75430054, 75430055, 75430056, 75430057]

In [20]:
idx = np.where(np.in1d(ccd['expnum'], expnum_list))[0]

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/results_20160603-more.pdf') as pdf:

    for ccd_index in idx:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

        ######################## Original mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(image_dir, fn_ood)
        
        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()
        
        ######################## New mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(mask_dir, fn_ood)
        
        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

9060 r 75430048
9061 r 75430049
9062 r 75430050
9063 r 75430051
9064 r 75430052
9065 r 75430053
9066 r 75430054
9067 r 75430055
9068 r 75430056
9069 r 75430057
